In [1]:
#initialization
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
import qiskit.quantum_info as qi
from numpy import pi
import os
import pandas as pd

In [8]:
def normal1D(N):
        a0 = 1 / np.sqrt(N)
        a1 = np.sqrt(2/N)

        result = np.zeros((N, N))

        for u in range(0, N):
            for n in range(0, N):
                coef = a1
                if u == 0:
                    coef = a0

                result[u, n] = coef * np.cos((2*n + 1) * u * pi / (2 * N))

        return result

def initState(qc, qubits, state):
    vec = state.flatten()
    norm = np.absolute(vec).sum()
    normState = []
    for i in range(0, vec.size):
        normState.append(float((1 if vec[i] >= 0 else -1) * np.sqrt(np.absolute(vec[i]))) / np.sqrt(norm))
    qc.initialize(normState, qubits)
    return norm
    
def createBlockFromMeasurements(measurements, totalMeasurements, size, norm):
    block = np.array([[0] * size] * size)
    for (key, value) in measurements.items():
        index = int(key, 2)
#         print(int(index/size))
#         print(int(index % size))
              
#         print((50 * (value / totalMeasurements)))
        block[int(index / size)][index % size] =  int(norm) * (value / totalMeasurements)
#         print(block)
    return block
#     print(block)
    
def qctIteration(qc, qubits, size):
    qct = qi.Operator(np.array(normal1D(2**int(size/2))))
    qc.unitary(qct, range(0,int(size/2)), label="qct")
    qc.unitary(qct, range(int(size/2), size), label="qct")
    
    

In [ ]:
state = np.array([[42, 7, 12, 8],
                         [10, 1, 15, 2],
                         [10, -24, -11, 10],
                         [10, -9, -10, -42]
                         ])
totalMeasurements = 6
size = 6
N = 2**int(size/2)
testOutput = {'00': 255, '10': 269, '01': 252, '11': 248}
dctLossy = normal1D(N)

path = os.path.abspath("deers.jpg")
image = ImageReader.getImage(path, N)
# ImageReader.displayImage(image)
chopped = ImageReader.chopUpImage(image, N)
# qr = QuantumRegister(size, 'input')
# cr = ClassicalRegister(size, 'result')
# qc = QuantumCircuit(qr, cr)

# initState(qc, qr, state)
# qc.barrier()
# qctIteration(qc, qr, size)
# qc.measure(qr, cr)
# qc.draw()

# # Use Aer's qasm_simulator
# backend_sim = Aer.get_backend('qasm_simulator')

# # Execute the circuit on the qasm simulator.
# # We've set the number of repeats of the circuit
# # to be 1024, which is the default.
# job_sim = backend_sim.run(transpile(qc, backend_sim), shots=totalMeasurements)

# # Grab the results from the job.
# result_sim = job_sim.result()
# counts = result_sim.get_counts(qc)
# # print(counts)
# createBlockFromMeasurements(counts, totalMeasurements, size)


total = 0
print(len(chopped['pieces']))
for i in range(0, len(chopped['pieces'])):
#     print(i)
    state = chopped['pieces'][i]
    qr = QuantumRegister(size, 'input')
    cr = ClassicalRegister(size, 'result')
    qc = QuantumCircuit(qr, cr)

    norm = initState(qc, qr, state)
    qc.barrier()
    qctIteration(qc, qr, size)
    qc.measure(qr, cr)
    qc.draw()

    # Use Aer's qasm_simulator
    backend_sim = Aer.get_backend('qasm_simulator')

    # Execute the circuit on the qasm simulator.
    # We've set the number of repeats of the circuit
    # to be 1024, which is the default.
    job_sim = backend_sim.run(transpile(qc, backend_sim), shots=totalMeasurements)

    # Grab the results from the job.
    result_sim = job_sim.result()
    counts = result_sim.get_counts(qc)
    # print(counts)
    
    chopped['pieces'][i] = createBlockFromMeasurements(counts, totalMeasurements, N, norm)
    chopped['pieces'][i] = DCT.remakeImageImage(
        chopped['pieces'][i], dctLossy)
# print(chopped)
reconstructed = ImageReader.reconstructImage(chopped, N)
print(image)
print(reconstructed)
ImageReader.displayImage( image, "Original")
ImageReader.displayImage( reconstructed, "Compressed")

cv2.waitKey(0)
cv2.destroyAllWindows()


6100
[[144  99  88 ... 104 117 127]
 [148 107  94 ... 108 118 125]
 [145 111  95 ... 120 127 129]
 ...
 [105  96  89 ...  82  74  84]
 [122 111 101 ...  79  71  84]
 [134 123 110 ...  75  68  83]]
[[255 185  51 ... 255 244 237]
 [255 209  75 ... 255 244 237]
 [255 242 108 ... 255 244 237]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [3]:
class DCT:

    def normal1D(N):
        a0 = 1 / math.sqrt(N)
        a1 = math.sqrt(2/N)

        result = np.zeros((N, N))

        for u in range(0, N):
            for n in range(0, N):
                coef = a1
                if u == 0:
                    coef = a0

                result[u, n] = coef * np.cos((2*n + 1) * u * pi / (2 * N))

        return result

    def getDCTBitcount(dct):
        max = np.amax(dct)
        return int(math.log2(max)) + 1

    def int1D(N, precision):
        a0 = 1 / math.sqrt(N)
        a1 = math.sqrt(2/N)

        result = np.zeros((N, N), dtype=int)

        for u in range(0, N):
            for n in range(0, N):
                coef = a1
                if u == 0:
                    coef = a0

                result[u, n] = int(
                    (2**precision) * coef * np.cos((2*n + 1) * u * pi / (2 * N)))

        return result

    def convertImage(image, dct, precision=None):
        if (precision):
            return (np.matmul(np.matmul(dct, image).astype(int) / 2**(precision), np.transpose(dct)).astype(int) / 2**(precision))
        return np.matmul(np.matmul(dct, image), np.transpose(dct))

    def convertImage1D(image, dct, precision=None):
        return np.matmul(dct, image)

    def convertImage2D(image, dct, precision=None):
        return np.matmul(np.matmul(dct, image), np.transpose(dct))

    def remakeImageImage(image, dct, precision=None):
        if (precision):
            return (np.matmul(np.matmul(np.transpose(dct).astype(int) / 2**(precision), image), dct).astype(int) / 2**(precision))
        return np.matmul(np.matmul(np.transpose(dct), image), dct)

    def removeSmallBits(chunk):
        chunk[abs(chunk) < 100] = 0
        return chunk


class ImageReader:

    def getImage(path, N):
        # Read Images
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        maxSize = tuple([(int(x) - (int(x) % N)) for x in gray.shape])
        maxSize = tuple(reversed(maxSize))
        return cv2.resize(gray, maxSize)

    def displayImage(image, name):
        cv2.imshow(name, image)

    def chopUpImage(image, N):
        result = []
        size = image.shape

        for i in range(0, size[0], N):
            for j in range(0, size[1], N):
                result.append((image[i:i+N, j:j+N]).astype(int) - 128)

        return {
            "pieces": result,
            "size": size
        }

    def reconstructImage(pieces, N):
        length = pieces['size'][0]
        width = pieces['size'][1]
        vfunc = np.vectorize(setLimit)
        result = np.ndarray(shape=(length, width), dtype=np.uint8)
        index = 0
        for i in range(0, length, N):
            for j in range(0, width, N):
                # print((pieces['pieces'][index] + 128))
                result[i:i+N, j:j+N] = vfunc(pieces['pieces'][index] + 128)
                # print(result[i:i+N, j:j+N], i, j)
                index += 1
        
        return result
        
def setLimit(a):
    a = a if a < 255 else 255
    return a if a > -255 else -255